In [29]:
import pandas as pd

In [39]:
results_df = pd.read_csv('1976-2016-president.csv')


In [40]:
results_df.head(10)

years = [1992,1996,2000,2004,2008,2012]
parties = ['democrat', 'republican']

results_df = results_df[results_df.year.isin(years)]
results_df = results_df[results_df.party.isin(parties)]

##Some duplicate entries (probably faithless elector votes)
#results_df[(results_df['year']==2004) & (results_df['state_po'] == 'MD')]
#results_df[(results_df['year']==2016) & (results_df['state_po'] == 'AZ')]
#results_df[(results_df['year']==2016) & (results_df['state_po'] == 'MD')]

In [41]:
results_df.head(10)

results_df['vote_share'] = round(100 * (results_df['candidatevotes']/results_df['totalvotes']),2) 
results_df = results_df[results_df['vote_share'] > 1] ##There were a few people who voted for Sanders/Kasich in 2016

In [42]:
results_df.head(10)


results_df = results_df.filter(items = ['year','state_po','party','vote_share'])
results_df = results_df.set_index(['year','state_po','party'],verify_integrity=True)


In [43]:
results_df.head(10)

results_df= results_df.unstack(level='party')
results_df['vote_diff'] = results_df['vote_share']['democrat'] - results_df['vote_share']['republican'] 

In [44]:
results_df.head(10)

partisan_lean_by_state = results_df.groupby('state_po')['vote_diff'].mean().to_frame()
partisan_lean_by_state.sort_values(by=['vote_diff'])

,vote_diff
state_po,
UT,-33.670000
WY,-28.660000
ID,-27.856667
NE,-22.470000
OK,-22.383333
AK,-19.786667
KS,-17.701667
ND,-17.010000
AL,-16.338333
